In [1]:
import torch 
from torch import nn 
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets,transforms
import os 

print(torch.__version__)

2.5.1+cu118


In [2]:
if torch.cuda.is_available():
    device='cuda'
elif torch.backends.mps.is_available(): 
    device=torch.device("mps")
else:
    device="cpu"

print(f"using device ={device}")

using device =cuda


In [3]:
train_dir="CustomDataSet/data/train"
test_dir="CustomDataSet/data/test"

In [4]:
# set up transformers 
train_transforms = transforms.Compose([
    transforms.Resize((128,128)), 
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.RandomGrayscale(p=0.1),
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

test_transforms = transforms.Compose([
    transforms.Resize((128,128)), 
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])


In [5]:
# load datasets 
train_data = datasets.ImageFolder(root=train_dir,transform=train_transforms)
test_data = datasets.ImageFolder(root=test_dir,transform=test_transforms)



In [6]:
# create data loader 
batch_size=32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=False)



In [7]:
class_name = train_data.classes
print(class_name)

['donuts', 'french_fries', 'fried_rice', 'samosa']


In [8]:
class_name_idx = train_data.class_to_idx
print(class_name_idx)

{'donuts': 0, 'french_fries': 1, 'fried_rice': 2, 'samosa': 3}


In [9]:
from model import CustomCNN 
num_classes = len(train_data.classes)
model_0=CustomCNN(num_classes=num_classes).to(device)

In [10]:
# lets define loss functiojn 
criterion = nn.CrossEntropyLoss()
#define optimezer 
optimizer = torch.optim.Adam(params=model_0.parameters(),lr=0.001,weight_decay=1e-4)

In [ ]:
from trainNN    import train
torch.manual_seed(42)
from timeit import default_timer as timer
start_time = timer()
results = train(
    model=model_0,
    train_dataloader=train_loader, 
    test_dataloader=test_loader, 
    optimizer=optimizer , 
    loss_fn=criterion, 
    epochs=10, 
    device=device
)

end_time=timer()
print(f"Total training  time is {end_time - start_time} seconds ")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.3583 | train_acc: 0.3241 | test_loss: 1.2622 | test_acc: 0.4013
Epoch: 2 | train_loss: 1.2594 | train_acc: 0.4008 | test_loss: 1.2184 | test_acc: 0.4293
Epoch: 3 | train_loss: 1.2539 | train_acc: 0.4023 | test_loss: 1.2013 | test_acc: 0.4803
Epoch: 4 | train_loss: 1.1915 | train_acc: 0.4764 | test_loss: 1.1707 | test_acc: 0.4655
Epoch: 5 | train_loss: 1.1562 | train_acc: 0.4971 | test_loss: 1.1351 | test_acc: 0.4967
Epoch: 6 | train_loss: 1.1072 | train_acc: 0.5129 | test_loss: 1.0618 | test_acc: 0.5576
Epoch: 7 | train_loss: 1.0670 | train_acc: 0.5544 | test_loss: 1.0816 | test_acc: 0.5428
Epoch: 8 | train_loss: 1.0205 | train_acc: 0.5801 | test_loss: 1.0600 | test_acc: 0.5855
Epoch: 9 | train_loss: 0.9712 | train_acc: 0.6051 | test_loss: 1.0635 | test_acc: 0.5872


In [ ]:
# from helper_functions import plot_loss_curves 
# plot_loss_curves(results)

In [ ]:
from pathlib import Path 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME="Modelv1.pth"
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME
print(f" Saving model to { MODEL_SAVE_PATH}")
torch.save(obj=model_0.state_dict(),f=MODEL_SAVE_PATH)

In [ ]:

# pip install torchinfo 
from torchinfo import summary 
summary(model=model_0,
       input_size=(32,3,128,128),
        col_names=("input_size","num_params","trainable")
        col_width=20, 
        row_settings=["var_names"]
       )